**Amount Spent per Customer**

In [1]:
SELECT customer_id, SUM(price) total_sales
FROM sales s JOIN menu m
ON s.product_id = m.product_id
GROUP BY customer_id
ORDER BY total_sales

(3 rows affected)

Total execution time: 00:00:00.121

customer_id,total_sales
C,36
B,74
A,76


**Time Spent per Customer**

In [2]:
SELECT customer_id, COUNT(DISTINCT order_date) total_visits
FROM sales
GROUP BY customer_id
ORDER BY total_visits

(3 rows affected)

Total execution time: 00:00:00.020

customer_id,total_visits
C,2
A,4
B,6


**First item purchased by each customer**

In [3]:
SELECT DISTINCT customer_id, order_date, product_name
FROM sales sales JOIN menu menu
ON sales.product_id = menu.product_id
WHERE order_date = '2021-01-01'
ORDER BY order_date 

(4 rows affected)

Total execution time: 00:00:00.087

customer_id,order_date,product_name
A,2021-01-01,curry
A,2021-01-01,sushi
B,2021-01-01,curry
C,2021-01-01,ramen


**Most popular item**

In [4]:
SELECT top 1 product_name Most_Purchased_Meal, COUNT(product_name) Times_a_Meal_was_Purchased
FROM menu menu JOIN sales sales
ON menu.product_id = sales.product_id
GROUP BY product_name
ORDER BY Times_a_Meal_was_Purchased DESC

(1 row affected)

Total execution time: 00:00:00.029

Most_Purchased_Meal,Times_a_Meal_was_Purchased
ramen,8


**Times Customers Purchased Ramen**

In [5]:
SELECT customer_id, product_name, COUNT(product_name) ramen_count
FROM menu menu JOIN sales sales
ON menu.product_id = sales.product_id
WHERE product_name = 'Ramen'
GROUP BY customer_id, product_name

(3 rows affected)

Total execution time: 00:00:00.021

customer_id,product_name,ramen_count
A,ramen,3
B,ramen,2
C,ramen,3


**Most popular item per customer**

In [6]:
SELECT DISTINCT customer_id, product_name, COUNT(sales.product_id) order_count
FROM sales sales JOIN menu menu
ON sales.product_id = menu.product_id
WHERE NOT customer_id = 'A' OR NOT sales.product_id IN (1, 2)
GROUP BY customer_id, product_name
ORDER BY customer_id, order_count DESC

(5 rows affected)

Total execution time: 00:00:00.017

customer_id,product_name,order_count
A,ramen,3
B,ramen,2
B,curry,2
B,sushi,2
C,ramen,3


**First Item Purchased by Customers after becoming members**

In [7]:
WITH member_sales AS (
SELECT s.customer_id, order_date, product_name,
DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY order_date) rank
FROM sales s JOIN menu m
ON s.product_id = m.product_id JOIN members m2 
ON s.customer_id = m2.customer_id
WHERE order_date > join_date
)
SELECT customer_id, product_name
FROM member_sales
WHERE rank = 1

(2 rows affected)

Total execution time: 00:00:00.021

customer_id,product_name
A,ramen
B,sushi


**First items purchased by members before becoming members**

In [8]:
WITH member_sales AS (
SELECT s.customer_id, product_name, order_date,
DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY order_date DESC) rank
FROM sales s JOIN menu m
ON s.product_id = m.product_id JOIN members m2
ON s.customer_id = m2.customer_id
WHERE order_date < join_date
)
SELECT DISTINCT customer_id, product_name, order_date
FROM member_sales
WHERE rank = 1

(3 rows affected)

Total execution time: 00:00:00.022

customer_id,product_name,order_date
A,curry,2021-01-01
A,sushi,2021-01-01
B,sushi,2021-01-04


**Number of items purchased and amount spent by members before their membersips**

In [9]:
WITH total_cte AS
(SELECT s.customer_id, JOIN_date, order_date, product_id
FROM sales s JOIN members m
ON s.customer_id = m.customer_id
WHERE order_date < JOIN_date)
SELECT customer_id, SUM(cte.product_id) total_items_purhcased, SUM(price) total_amount
FROM total_cte cte JOIN menu m
	on cte.product_id = m.product_id
GROUP BY cte.customer_id

(2 rows affected)

Total execution time: 00:00:00.023

customer_id,total_items_purhcased,total_amount
A,3,25
B,5,40


**Points earned per customer ($1.00 = 10 points & Sushi has 2x points mulitplyer)**

In [10]:
SELECT customer_id,
SUM(
CASE
	WHEN sales.product_id in (2, 3) THEN (price * 10)
	ELSE (price * 20)
END) points
FROM sales sales JOIN menu menu
ON sales.product_id = menu.product_id
GROUP BY customer_id
ORDER BY points

(3 rows affected)

Total execution time: 00:00:00.017

customer_id,points
C,360
A,860
B,940


**Points earned per Customer if all items give 2x on the first week of membership**

In [11]:
WITH cte as (
SELECT customer_id
, join_date
, DATEADD(DAY, 6, join_date) AS day_7
FROM members
)
SELECT a.customer_id, 
SUM(CASE
    WHEN product_name = 'sushi' THEN price * 20
    WHEN order_date BETWEEN join_date AND day_7 THEN price * 20
    ELSE price * 10 END) AS points
FROM sales a
JOIN cte 
 ON a.customer_id = cte.customer_id
JOIN menu c
 ON a.product_id = c.product_id
 where order_date <= '2021-01-31'
GROUP BY a.customer_id

(2 rows affected)

Total execution time: 00:00:00.018

customer_id,points
A,1370
B,820
